In [1]:
# ============ 1. ИМПОРТ БИБЛИОТЕК ============
import asyncio
import aiohttp
import json
from datetime import datetime, timedelta
import os
import sys
import hashlib
import numpy as np
import subprocess
import glob
import time
import io
import re
import pickle
from typing import List, Dict, Any, Optional, Set
from dataclasses import dataclass, asdict, field
from collections import defaultdict
import warnings
from pathlib import Path
import logging

# Настройка логирования
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Отключаем предупреждения
warnings.filterwarnings('ignore')
os.environ['HF_HUB_DISABLE_SYMLINKS_WARNING'] = '1'

# Функция для установки библиотек
def install_packages():
    """Установка необходимых библиотек"""
    required = ['aiohttp', 'numpy', 'scikit-learn', 'Pillow', 'requests']
    print(" Проверяем библиотеки...")
    for package in required:
        try:
            __import__(package.replace('-', '_'))
        except ImportError:
            print(f" Устанавливаем {package}...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", package, "-q"])

install_packages()

# Дополнительные импорты после установки
try:
    import requests
    from PIL import Image, ImageFile
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.metrics.pairwise import cosine_similarity
    from sklearn.preprocessing import normalize
except ImportError as e:
    print(f" Ошибка импорта библиотек: {e}")
    print(" Пробуем установить заново...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "scikit-learn", "Pillow", "requests", "-q"])
    import requests
    from PIL import Image, ImageFile
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.metrics.pairwise import cosine_similarity
    from sklearn.preprocessing import normalize

# ============ 2. КОНСТАНТЫ И НАСТРОЙКИ ============
CATEGORIES = [
    ('concert', 'Концерты'),
    ('theater', 'Театр'),
    ('exhibition', 'Выставки'),
    ('festival', 'Фестивали'),
    ('education', 'Образование'),
    ('party', 'Вечеринки'),
    ('tour', 'Экскурсии'),
    ('entertainment', 'Развлечения')
]

CITIES = {
    'msk': {
        'name': 'Москва',
        'timezone': 'Europe/Moscow',
        'center': {'lat': 55.7558, 'lon': 37.6173}
    },
    'spb': {
        'name': 'Санкт-Петербург',
        'timezone': 'Europe/Moscow',
        'center': {'lat': 59.9343, 'lon': 30.3351}
    }
}

# ============ 3. МОДЕЛИ ДАННЫХ ============
@dataclass
class PlaceInfo:
    """Детальная информация о месте проведения"""
    id: Optional[int] = None
    title: str = ""
    address: str = ""
    subway: str = ""
    coords: Optional[Dict[str, float]] = None
    phone: str = ""
    site_url: str = ""
    description: str = ""
    is_stub: bool = False
    
    def to_text(self) -> str:
        """Преобразование в текстовое описание"""
        parts = []
        if self.title and self.title != "Место не указано":
            parts.append(f"Место: {self.title}")
        if self.address:
            parts.append(f"Адрес: {self.address}")
        if self.subway:
            # Убедимся, что subway - строка
            if isinstance(self.subway, list):
                subway = ', '.join([str(s) for s in self.subway[:3]])
            else:
                subway = str(self.subway)
            parts.append(f"Метро: {subway}")
        
        if self.coords and 'lat' in self.coords and 'lon' in self.coords:
            lat = self.coords['lat']
            lon = self.coords['lon']
            if lat is not None and lon is not None:
                try:
                    lat_val = float(lat) if not isinstance(lat, (int, float)) else lat
                    lon_val = float(lon) if not isinstance(lon, (int, float)) else lon
                    parts.append(f"Координаты: {lat_val:.6f}, {lon_val:.6f}")
                except (ValueError, TypeError, AttributeError):
                    pass
        
        if self.phone:
            parts.append(f"Телефон: {self.phone}")
        if self.site_url:
            parts.append(f"Сайт: {self.site_url}")
        if self.description:
            short_desc = self.description[:150] + "..." if len(self.description) > 150 else self.description
            parts.append(f"Описание: {short_desc}")
        
        return "\n".join(parts) if parts else "Место не указано"

@dataclass
class DateInfo:
    """Информация о дате события"""
    start_timestamp: Optional[int] = None
    end_timestamp: Optional[int] = None
    start_datetime: str = ""
    end_datetime: str = ""
    year: int = 0
    month: int = 0
    day: int = 0
    hour: int = 0
    minute: int = 0
    
    def to_text(self) -> str:
        """Преобразование в текстовое представление"""
        if not self.start_datetime:
            return "Дата не указана"
        
        try:
            if 'Z' in self.start_datetime:
                start_dt = datetime.fromisoformat(self.start_datetime.replace('Z', '+00:00'))
            else:
                start_dt = datetime.fromisoformat(self.start_datetime)
                
            date_str = start_dt.strftime('%d.%m.%Y %H:%M')
            
            if self.end_datetime:
                if 'Z' in self.end_datetime:
                    end_dt = datetime.fromisoformat(self.end_datetime.replace('Z', '+00:00'))
                else:
                    end_dt = datetime.fromisoformat(self.end_datetime)
                    
                if start_dt.date() == end_dt.date():
                    date_str += f" - {end_dt.strftime('%H:%M')}"
                else:
                    date_str += f" - {end_dt.strftime('%d.%m.%Y %H:%M')}"
            
            return date_str
        except (ValueError, TypeError, AttributeError):
            return "Дата не указана"

@dataclass
class PriceInfo:
    """Информация о цене"""
    is_free: bool = False
    min_price: Optional[float] = None
    max_price: Optional[float] = None
    currency: str = "руб."
    description: str = ""
    
    def to_text(self) -> str:
        """Преобразование в текстовое представление"""
        if self.is_free:
            return "Бесплатно"
        
        try:
            if self.min_price is not None and self.max_price is not None:
                min_val = float(self.min_price) if not isinstance(self.min_price, (int, float)) else self.min_price
                max_val = float(self.max_price) if not isinstance(self.max_price, (int, float)) else self.max_price
                
                if min_val == max_val:
                    return f"{int(min_val)} {self.currency}"
                else:
                    return f"от {int(min_val)} до {int(max_val)} {self.currency}"
            elif self.min_price is not None:
                min_val = float(self.min_price) if not isinstance(self.min_price, (int, float)) else self.min_price
                return f"от {int(min_val)} {self.currency}"
            elif self.max_price is not None:
                max_val = float(self.max_price) if not isinstance(self.max_price, (int, float)) else self.max_price
                return f"до {int(max_val)} {self.currency}"
        except (ValueError, TypeError, AttributeError):
            pass
        
        if self.description:
            return self.description
        
        return "Цена не указана"

@dataclass
class Event:
    """Основная модель события"""
    # Основная информация
    id: int
    title: str
    description: str = ""
    short_description: str = ""
    category: str = ""
    
    # Временные параметры
    dates: List[DateInfo] = field(default_factory=list)
    dates_text: List[str] = field(default_factory=list)
    
    # Финансовые параметры
    price: PriceInfo = field(default_factory=PriceInfo)
    price_text: str = ""
    
    # Местоположение
    place: PlaceInfo = field(default_factory=PlaceInfo)
    place_text: str = ""
    
    # Метаданные
    url: str = ""
    tags: List[str] = field(default_factory=list)
    images: List[Dict] = field(default_factory=list)
    image_count: int = 0
    age_restriction: Optional[str] = None
    participants: List[str] = field(default_factory=list)
    
    # География
    city: str = ""
    city_name: str = ""
    
    # Технические поля
    parsed_at: str = ""
    source: str = "kudago"
    is_real_data: bool = True
    
    # Для поиска
    embedding_text: str = ""
    
    def __post_init__(self):
        # Инициализация списков
        if self.tags is None:
            self.tags = []
        if self.images is None:
            self.images = []
        if self.participants is None:
            self.participants = []
        if self.dates is None:
            self.dates = []
        
        # Безопасное создание текстовых представлений
        try:
            self.place_text = self.place.to_text()
        except Exception as e:
            logger.warning(f"Ошибка при создании place_text: {e}")
            self.place_text = "Место не указано"
        
        try:
            self.price_text = self.price.to_text()
        except Exception as e:
            logger.warning(f"Ошибка при создании price_text: {e}")
            self.price_text = "Цена не указана"
        
        # Безопасное создание dates_text
        self.dates_text = []
        for date in self.dates:
            try:
                text = date.to_text()
                if text and text != "Дата не указана":
                    self.dates_text.append(text)
            except Exception as e:
                logger.warning(f"Ошибка при создании dates_text: {e}")
                continue
        
        # Безопасное создание embedding_text
        try:
            self.embedding_text = self._generate_embedding_text_safe()
        except Exception as e:
            logger.warning(f"Ошибка при создании embedding_text: {e}")
            self.embedding_text = f"{self.title} | {self.category} | {self.city_name}"
    
    def _generate_embedding_text_safe(self) -> str:
        """Безопасная генерация текста для эмбеддингов"""
        parts = []
        
        try:
            if self.title:
                parts.append(f"Название: {self.title}")
            
            if self.category:
                parts.append(f"Категория: {self.category}")
            
            if self.description:
                clean_desc = re.sub(r'<[^>]+>', ' ', self.description)
                clean_desc = re.sub(r'\s+', ' ', clean_desc).strip()
                if clean_desc:
                    parts.append(f"Описание: {clean_desc[:300]}")
            
            if self.place_text and self.place_text != "Место не указано":
                place_first_line = self.place_text.split('\n')[0] if '\n' in self.place_text else self.place_text
                parts.append(f"Место: {place_first_line[:100]}")
            
            if self.tags:
                # Убедимся, что теги - строки
                tag_strings = []
                for tag in self.tags[:5]:
                    if isinstance(tag, str):
                        tag_strings.append(tag)
                    elif isinstance(tag, dict):
                        # Если тег - словарь, попробуем извлечь значение
                        for key in ['name', 'title', 'value']:
                            if key in tag and isinstance(tag[key], str):
                                tag_strings.append(tag[key])
                                break
                        else:
                            tag_strings.append(str(tag))
                    else:
                        tag_strings.append(str(tag))
                
                if tag_strings:
                    parts.append(f"Теги: {', '.join(tag_strings)}")
            
            if self.dates_text:
                parts.append(f"Дата: {self.dates_text[0]}")
            
            if self.city_name:
                parts.append(f"Город: {self.city_name}")
        except Exception as e:
            logger.warning(f"Ошибка в _generate_embedding_text_safe: {e}")
            return f"{self.title} | {self.category}"
        
        return " | ".join(parts) if parts else f"{self.title} | {self.category}"
    
    def to_dict(self) -> Dict[str, Any]:
        """Преобразование в словарь с обработкой ошибок"""
        try:
            result = asdict(self)
            
            # Обрабатываем вложенные объекты
            result['dates'] = []
            for date in self.dates:
                try:
                    result['dates'].append(asdict(date))
                except Exception:
                    continue
            
            try:
                result['price'] = asdict(self.price) if self.price else {}
            except Exception:
                result['price'] = {}
            
            try:
                result['place'] = asdict(self.place) if self.place else {}
            except Exception:
                result['place'] = {}
            
            # Преобразуем теги в строки
            if 'tags' in result and result['tags']:
                cleaned_tags = []
                for tag in result['tags']:
                    if isinstance(tag, str):
                        cleaned_tags.append(tag)
                    elif isinstance(tag, dict):
                        # Если тег - словарь, попробуем извлечь строку
                        for key in ['name', 'title', 'value']:
                            if key in tag and isinstance(tag[key], str):
                                cleaned_tags.append(tag[key])
                                break
                        else:
                            cleaned_tags.append(str(tag))
                    else:
                        cleaned_tags.append(str(tag))
                result['tags'] = cleaned_tags
            
            return result
        except Exception as e:
            logger.error(f"Ошибка при преобразовании события в словарь: {e}")
            return {
                'id': self.id,
                'title': self.title,
                'category': self.category,
                'city': self.city,
                'city_name': self.city_name,
                'parsed_at': self.parsed_at
            }

# ============ 4. УСТОЙЧИВЫЙ ПАРСЕР KUDAGO ============
class RobustKudaGoParser:
    """Устойчивый парсер событий с обработкой ошибок и задержками"""
    
    def __init__(self, city: str = 'msk'):
        self.base_url = "https://kudago.com/public-api/v1.4"
        self.city = city
        self.city_info = CITIES.get(city, {'name': city, 'timezone': 'Europe/Moscow'})
        self.session = None
        self.place_cache = {}
        self.request_delay = 2  # Задержка между запросами в секундах
        self.max_retries = 3   # Максимальное количество попыток
        
    async def __aenter__(self):
        self.session = aiohttp.ClientSession(
            headers={
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36',
                'Accept': 'application/json',
                'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7'
            },
            timeout=aiohttp.ClientTimeout(total=120)  # Увеличиваем таймаут
        )
        return self
    
    async def __aexit__(self, exc_type, exc_val, exc_tb):
        if self.session:
            await self.session.close()
    
    async def _safe_request(self, url: str, params: Dict = None, retry_count: int = 0):
        """Безопасный запрос с повторными попытками"""
        try:
            await asyncio.sleep(self.request_delay)  # Задержка между запросами
            
            async with self.session.get(
                url,
                params=params,
                raise_for_status=False
            ) as response:
                if response.status == 200:
                    return await response.json()
                elif response.status in [429, 503]:  # Слишком много запросов или сервис недоступен
                    if retry_count < self.max_retries:
                        wait_time = (retry_count + 1) * 5  # Экспоненциальная задержка
                        logger.warning(f"Сервер вернул {response.status}, повторная попытка через {wait_time} сек...")
                        await asyncio.sleep(wait_time)
                        return await self._safe_request(url, params, retry_count + 1)
                    else:
                        logger.error(f"Сервер недоступен после {self.max_retries} попыток")
                        return None
                else:
                    logger.warning(f"Ошибка HTTP {response.status} для URL: {url}")
                    return None
        except asyncio.TimeoutError:
            if retry_count < self.max_retries:
                logger.warning(f"Таймаут, повторная попытка {retry_count + 1}/{self.max_retries}")
                return await self._safe_request(url, params, retry_count + 1)
            else:
                logger.error("Таймаут после всех попыток")
                return None
        except Exception as e:
            logger.error(f"Ошибка при запросе: {e}")
            return None
    
    def _parse_place_info(self, place_data: Dict) -> PlaceInfo:
        """Парсинг информации о месте"""
        if not place_data or not isinstance(place_data, dict):
            return PlaceInfo(title="Место не указано")
        
        # Извлекаем координаты с проверкой на None
        coords = None
        location = place_data.get('location', {})
        if location and isinstance(location, dict):
            lat = location.get('lat')
            lon = location.get('lon')
            if lat is not None and lon is not None:
                try:
                    lat_val = float(lat) if not isinstance(lat, (int, float)) else lat
                    lon_val = float(lon) if not isinstance(lon, (int, float)) else lon
                    coords = {'lat': lat_val, 'lon': lon_val}
                except (ValueError, TypeError):
                    coords = None
        elif place_data.get('coords'):
            try:
                coords = place_data['coords']
            except Exception:
                coords = None
        
        # Обрабатываем метро
        subway = ""
        subway_data = place_data.get('subway')
        if subway_data:
            if isinstance(subway_data, list):
                subway_names = []
                for item in subway_data:
                    if isinstance(item, dict) and 'name' in item:
                        subway_names.append(str(item['name']))
                    elif isinstance(item, str):
                        subway_names.append(item)
                subway = ', '.join(subway_names[:3])  # Берем до 3 станций
            elif isinstance(subway_data, str):
                subway = subway_data
        
        return PlaceInfo(
            id=place_data.get('id'),
            title=place_data.get('title', 'Место не указано'),
            address=place_data.get('address', ''),
            subway=subway,
            coords=coords,
            phone=place_data.get('phone', ''),
            site_url=place_data.get('site_url', ''),
            description=place_data.get('description', ''),
            is_stub=place_data.get('is_stub', False)
        )
    
    def _parse_date_info_safe(self, date_data: Dict) -> Optional[DateInfo]:
        """Безопасный парсинг информации о дате"""
        if not date_data or not isinstance(date_data, dict):
            return None
        
        start = date_data.get('start')
        if not start:
            return None
        
        try:
            # Проверяем, что timestamp валидный
            if isinstance(start, (int, float)):
                # Проверяем, что дата не в далеком прошлом или будущем
                current_time = time.time()
                if start < 0 or start > current_time + 10 * 365 * 24 * 60 * 60:  # Не дальше 10 лет вперед
                    return None
                
                start_dt = datetime.fromtimestamp(start)
            else:
                return None
            
            end = date_data.get('end')
            end_dt = None
            
            if end and isinstance(end, (int, float)):
                if 0 < end < current_time + 10 * 365 * 24 * 60 * 60:
                    end_dt = datetime.fromtimestamp(end)
            
            return DateInfo(
                start_timestamp=start,
                end_timestamp=end,
                start_datetime=start_dt.isoformat(),
                end_datetime=end_dt.isoformat() if end_dt else "",
                year=start_dt.year,
                month=start_dt.month,
                day=start_dt.day,
                hour=start_dt.hour,
                minute=start_dt.minute
            )
        except (ValueError, TypeError, OSError) as e:
            logger.warning(f"Ошибка парсинга даты: {e}")
            return None
    
    def _parse_price_info_safe(self, price_data: Any) -> PriceInfo:
        """Безопасный парсинг информации о цене"""
        if isinstance(price_data, dict):
            min_price = None
            max_price = None
            
            # Пробуем извлечь числовые значения цены
            min_val = price_data.get('min')
            max_val = price_data.get('max')
            
            if min_val:
                try:
                    if isinstance(min_val, (int, float)):
                        min_price = float(min_val)
                    elif isinstance(min_val, str):
                        numbers = re.findall(r'\d+', min_val)
                        if numbers:
                            min_price = float(''.join(numbers))
                except (ValueError, TypeError):
                    pass
            
            if max_val:
                try:
                    if isinstance(max_val, (int, float)):
                        max_price = float(max_val)
                    elif isinstance(max_val, str):
                        numbers = re.findall(r'\d+', max_val)
                        if numbers:
                            max_price = float(''.join(numbers))
                except (ValueError, TypeError):
                    pass
            
            return PriceInfo(
                is_free=price_data.get('is_free', False),
                min_price=min_price,
                max_price=max_price,
                currency=price_data.get('currency', 'руб.'),
                description=str(price_data.get('description', ''))
            )
        elif isinstance(price_data, str):
            if price_data.lower() in ['бесплатно', 'free', '0']:
                return PriceInfo(is_free=True)
            else:
                return PriceInfo(description=price_data)
        else:
            return PriceInfo()
    
    async def _parse_event_safe(self, event_data: Dict, category: str) -> Optional[Event]:
        """Безопасный парсинг события из сырых данных"""
        try:
            # Проверяем обязательные поля
            title = event_data.get('title')
            if not title:
                return None
            
            # Парсим даты
            dates = []
            date_items = event_data.get('dates', [])
            if not isinstance(date_items, list):
                date_items = []
            
            for date_item in date_items:
                if isinstance(date_item, dict):
                    date_info = self._parse_date_info_safe(date_item)
                    if date_info:
                        dates.append(date_info)
            
            if not dates:
                return None
            
            # Проверяем, что хотя бы одна дата в будущем
            current_time = time.time()
            has_future_date = any(
                date.start_timestamp and date.start_timestamp > current_time 
                for date in dates
            )
            if not has_future_date:
                return None
            
            # Парсим место
            place_data = event_data.get('place', {})
            if not isinstance(place_data, dict):
                place_data = {}
            place_info = self._parse_place_info(place_data)
            
            # Парсим цену
            price_info = self._parse_price_info_safe(event_data.get('price'))
            
            # Обрабатываем описание
            description = event_data.get('description', '')
            if description:
                description = re.sub(r'<[^>]+>', ' ', description)
                description = re.sub(r'\s+', ' ', description).strip()
            
            # Обрабатываем изображения
            images = []
            image_items = event_data.get('images', [])
            if isinstance(image_items, list):
                for img in image_items[:3]:  # Берем до 3 изображений
                    if isinstance(img, dict) and img.get('image'):
                        images.append({
                            'url': str(img.get('image', '')),
                            'thumbnail': str(img.get('thumbnail', '')),
                            'source': 'kudago'
                        })
            
            # Обрабатываем теги
            tags = []
            tag_items = event_data.get('tags', [])
            if isinstance(tag_items, list):
                for tag in tag_items[:10]:  # Ограничиваем количество тегов
                    if isinstance(tag, str):
                        tags.append(tag)
                    elif isinstance(tag, dict):
                        # Извлекаем имя тега из словаря
                        for key in ['name', 'title', 'value']:
                            if key in tag and isinstance(tag[key], str):
                                tags.append(tag[key])
                                break
            
            # Создаем объект события
            event = Event(
                id=event_data.get('id', 0),
                title=str(title),
                description=description,
                short_description=description[:200] + "..." if len(description) > 200 else description,
                category=category,
                dates=dates,
                price=price_info,
                place=place_info,
                url=event_data.get('site_url', ''),
                tags=tags,
                images=images,
                image_count=len(images),
                age_restriction=event_data.get('age_restriction'),
                participants=event_data.get('participants', []),
                city=self.city,
                city_name=self.city_info['name'],
                parsed_at=datetime.now().isoformat(),
                source='kudago',
                is_real_data=True
            )
            
            return event
            
        except Exception as e:
            event_id = event_data.get('id', 'unknown')
            event_title = str(event_data.get('title', 'unknown'))[:50]
            logger.error(f"Ошибка при парсинге события (ID: {event_id}, Title: {event_title}...): {e}")
            return None
    
    async def fetch_category_events(self, category_code: str, category_name: str) -> List[Event]:
        """Загрузка событий по категории с обработкой ошибок"""
        print(f"  Загружаем {category_name}...")
        
        # Определяем временной диапазон
        today = datetime.now()
        start_date = today.strftime('%Y-%m-%d')
        end_date = '2025-12-31'
        
        params = {
            'categories': category_code,
            'location': self.city,
            'page_size': 50,  # Уменьшаем размер страницы
            'actual_since': start_date,
            'actual_until': end_date,
            'fields': ('id,title,description,dates,price,place,images,'
                      'site_url,tags,age_restriction,participants'),
            'expand': 'place',
            'text_format': 'text',
            'order_by': '-publication_date'
        }
        
        try:
            data = await self._safe_request(
                f"{self.base_url}/events/",
                params=params
            )
            
            if not data:
                print(f"      Не удалось загрузить данные для {category_name}")
                return []
            
            raw_events = data.get('results', [])
            if not isinstance(raw_events, list):
                raw_events = []
            
            # Парсим события
            events = []
            for raw_event in raw_events:
                if isinstance(raw_event, dict):
                    event = await self._parse_event_safe(raw_event, category_name)
                    if event:
                        events.append(event)
            
            print(f"      Найдено {len(events)} событий")
            return events
            
        except Exception as e:
            print(f"      Ошибка при загрузке {category_name}: {e}")
            return []
    
    async def get_all_events(self) -> List[Event]:
        """Получение всех событий для города"""
        print(f"\n{'='*60}")
        print(f" ПАРСИМ СОБЫТИЯ ДЛЯ {self.city_info['name'].upper()}")
        print(f"{'='*60}")
        
        all_events = []
        
        for cat_code, cat_name in CATEGORIES:
            events = await self.fetch_category_events(cat_code, cat_name)
            all_events.extend(events)
            
            # Задержка между категориями
            await asyncio.sleep(3)  # Увеличиваем задержку
        
        # Анализ результатов
        self._analyze_events(all_events)
        
        return all_events
    
    def _analyze_events(self, events: List[Event]):
        """Анализ собранных событий"""
        if not events:
            print("      Нет событий для анализа")
            return
        
        # Статистика по годам
        year_stats = defaultdict(int)
        for event in events:
            for date in event.dates:
                if date.year:
                    year_stats[date.year] += 1
        
        current_year = datetime.now().year
        print(f"\n      Анализ дат ({len(events)} событий):")
        for year in sorted(year_stats.keys()):
            count = year_stats[year]
            status = " ТЕКУЩИЙ" if year == current_year else " БУДУЩЕЕ" if year > current_year else " ПРОШЛОЕ"
            print(f"       {year}: {count} событий ({status})")
        
        # Статистика по категориям
        category_stats = defaultdict(int)
        for event in events:
            category_stats[event.category] += 1
        
        print(f"\n      Распределение по категориям:")
        for category, count in sorted(category_stats.items(), key=lambda x: x[1], reverse=True):
            print(f"       {category}: {count}")
        
        # Статистика по местам
        places_with_coords = sum(1 for e in events if e.place and e.place.coords)
        print(f"\n      Места с координатами: {places_with_coords}/{len(events)}")
        
        # Статистика по ценам
        free_events = sum(1 for e in events if e.price and e.price.is_free)
        print(f"      Бесплатных событий: {free_events}/{len(events)}")

# ============ 5. ЭКСПОРТЕР В JSON ============
class JSONExporter:
    """Экспорт данных в структурированные JSON файлы"""
    
    def __init__(self, base_dir: str = "./parsed_events"):
        self.base_dir = Path(base_dir)
        self.base_dir.mkdir(parents=True, exist_ok=True)
        
        # Создаем подпапки
        self.dirs = {
            'by_city': self.base_dir / "by_city",
            'by_category': self.base_dir / "by_category",
            'by_month': self.base_dir / "by_month",
            'raw': self.base_dir / "raw_data"
        }
        
        for dir_path in self.dirs.values():
            dir_path.mkdir(parents=True, exist_ok=True)
        
        print(f"  Создана структура папок в: {self.base_dir}")
    
    def export_events(self, events: List[Event], city: str):
        """Экспорт событий в JSON файлы"""
        if not events:
            print(f"  Нет событий для экспорта ({city})")
            return
        
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        # 1. Экспорт всех событий города
        self._export_city_events(events, city, timestamp)
        
        # 2. Экспорт по категориям
        self._export_by_category(events, city, timestamp)
        
        # 3. Экспорт по месяцам
        self._export_by_month(events, city, timestamp)
        
        # 4. Экспорт сырых данных
        self._export_raw_data(events, city, timestamp)
        
        print(f"  Данные успешно экспортированы для {city}")
    
    def _export_city_events(self, events: List[Event], city: str, timestamp: str):
        """Экспорт всех событий города в один файл"""
        city_data = {
            'metadata': {
                'city': city,
                'total_events': len(events),
                'export_date': datetime.now().isoformat(),
                'timestamp': timestamp
            },
            'events': [event.to_dict() for event in events]
        }
        
        filename = f"{city}_all_events_{timestamp}.json"
        filepath = self.dirs['by_city'] / filename
        
        with open(filepath, 'w', encoding='utf-8') as f:
            json.dump(city_data, f, ensure_ascii=False, indent=2, default=str)
        
        size_kb = os.path.getsize(filepath) // 1024
        print(f"    • Экспорт по городу: {filename} ({size_kb} KB)")
    
    def _export_by_category(self, events: List[Event], city: str, timestamp: str):
        """Экспорт событий по категориям"""
        # Группируем события по категориям
        events_by_category = defaultdict(list)
        for event in events:
            events_by_category[event.category].append(event.to_dict())
        
        for category, category_events in events_by_category.items():
            # Создаем безопасное имя для категории
            safe_category = re.sub(r'[^\w\s-]', '', category).replace(' ', '_').lower()
            
            category_data = {
                'metadata': {
                    'category': category,
                    'city': city,
                    'total_events': len(category_events),
                    'export_date': datetime.now().isoformat()
                },
                'events': category_events
            }
            
            filename = f"{safe_category}_{city}_{timestamp}.json"
            filepath = self.dirs['by_category'] / filename
            
            with open(filepath, 'w', encoding='utf-8') as f:
                json.dump(category_data, f, ensure_ascii=False, indent=2, default=str)
        
        print(f"    • Экспорт по категориям: {len(events_by_category)} файлов")
    
    def _export_by_month(self, events: List[Event], city: str, timestamp: str):
        """Экспорт событий по месяцам"""
        # Группируем события по месяцу
        events_by_month = defaultdict(list)
        for event in events:
            if event.dates:
                first_date = event.dates[0]
                month_key = f"{first_date.year}-{first_date.month:02d}"
                events_by_month[month_key].append(event.to_dict())
        
        for month_key, month_events in events_by_month.items():
            month_data = {
                'metadata': {
                    'month': month_key,
                    'city': city,
                    'total_events': len(month_events),
                    'export_date': datetime.now().isoformat()
                },
                'events': month_events
            }
            
            filename = f"{month_key}_{city}_{timestamp}.json"
            filepath = self.dirs['by_month'] / filename
            
            with open(filepath, 'w', encoding='utf-8') as f:
                json.dump(month_data, f, ensure_ascii=False, indent=2, default=str)
        
        print(f"    • Экспорт по месяцам: {len(events_by_month)} файлов")
    
    def _export_raw_data(self, events: List[Event], city: str, timestamp: str):
        """Экспорт сырых данных для анализа"""
        raw_data = {
            'metadata': {
                'city': city,
                'total_events': len(events),
                'parsed_at': datetime.now().isoformat(),
                'export_date': datetime.now().isoformat(),
                'data_version': '1.0',
                'source': 'KudaGo API'
            },
            'statistics': self._calculate_statistics(events),
            'events': [event.to_dict() for event in events]
        }
        
        filename = f"raw_data_{city}_{timestamp}.json"
        filepath = self.dirs['raw'] / filename
        
        with open(filepath, 'w', encoding='utf-8') as f:
            json.dump(raw_data, f, ensure_ascii=False, indent=2, default=str)
        
        size_mb = os.path.getsize(filepath) / (1024 * 1024)
        print(f"    • Сырые данные: {filename} ({size_mb:.2f} MB)")
    
    def _calculate_statistics(self, events: List[Event]) -> Dict:
        """Расчет статистики по событиям"""
        stats = {
            'total_events': len(events),
            'categories': {},
            'date_range': {'min': None, 'max': None},
            'price_stats': {'free_events': 0, 'paid_events': 0},
            'place_stats': {'with_coords': 0, 'with_images': 0}
        }
        
        if not events:
            return stats
        
        # Категории
        categories = defaultdict(int)
        for event in events:
            categories[event.category] += 1
        stats['categories'] = dict(categories)
        
        # Диапазон дат
        min_date = float('inf')
        max_date = float('-inf')
        for event in events:
            for date in event.dates:
                if date.start_timestamp:
                    min_date = min(min_date, date.start_timestamp)
                    max_date = max(max_date, date.start_timestamp)
        
        if min_date != float('inf'):
            stats['date_range']['min'] = datetime.fromtimestamp(min_date).strftime('%Y-%m-%d')
        if max_date != float('-inf'):
            stats['date_range']['max'] = datetime.fromtimestamp(max_date).strftime('%Y-%m-%d')
        
        # Цены
        for event in events:
            if event.price and event.price.is_free:
                stats['price_stats']['free_events'] += 1
            else:
                stats['price_stats']['paid_events'] += 1
        
        # Места
        for event in events:
            if event.place and event.place.coords:
                stats['place_stats']['with_coords'] += 1
            if event.image_count > 0:
                stats['place_stats']['with_images'] += 1
        
        return stats

# ============ 6. ОСНОВНОЙ ПРОЦЕСС ============
async def parse_and_export_city(city: str) -> Dict:
    """Парсинг и экспорт событий для одного города"""
    print(f"\n{'='*70}")
    print(f"   НАЧИНАЕМ ПАРСИНГ {CITIES[city]['name'].upper()}")
    print(f"{'='*70}")
    
    try:
        async with RobustKudaGoParser(city) as parser:
            events = await parser.get_all_events()
            
            if events:
                # Создаем экспортер
                exporter = JSONExporter(f"./parsed_events/{city}")
                exporter.export_events(events, city)
                
                return {
                    'city': city,
                    'events': events,
                    'count': len(events),
                    'success': True
                }
            else:
                print(f"  Не удалось получить события для {city}")
                return {'city': city, 'events': [], 'count': 0, 'success': False}
                
    except Exception as e:
        print(f"  Критическая ошибка при парсинге {city}: {e}")
        import traceback
        traceback.print_exc()
        return {'city': city, 'events': [], 'count': 0, 'success': False}

async def parse_multiple_cities(cities: List[str] = None) -> Dict:
    """Парсинг нескольких городов"""
    if cities is None:
        cities = ['msk', 'spb']
    
    print(f"\n{'='*70}")
    print("  ЗАПУСК ПАРСИНГА СОБЫТИЙ")
    print(f"{'='*70}")
    print(f" Период: сегодня - конец 2025 года")
    print(f" Города: {', '.join([CITIES[c]['name'] for c in cities])}")
    print(f"{'='*70}")
    
    all_results = {}
    
    for city in cities:
        result = await parse_and_export_city(city)
        all_results[city] = result
        
        if result['success']:
            print(f"  {city.upper()}: {result['count']} событий успешно экспортировано")
        else:
            print(f"  {city.upper()}: события не найдены или произошла ошибка")
        
        # Задержка между городами
        await asyncio.sleep(5)
    
    # Создаем сводный отчет
    create_summary_report(all_results)
    
    return all_results

def create_summary_report(all_results: Dict):
    """Создание сводного отчета"""
    print(f"\n{'='*70}")
    print("  СОЗДАНИЕ СВОДНОГО ОТЧЕТА")
    print(f"{'='*70}")
    
    total_events = 0
    summary_data = {
        'metadata': {
            'created_at': datetime.now().isoformat(),
            'total_cities': len(all_results),
            'data_version': '1.0'
        },
        'cities': {},
        'totals': {'events': 0}
    }
    
    for city, result in all_results.items():
        events = result.get('events', [])
        total_events += len(events)
        
        summary_data['cities'][city] = {
            'name': CITIES[city]['name'],
            'total_events': len(events),
            'success': result.get('success', False),
            'parsed_at': datetime.now().isoformat()
        }
    
    summary_data['totals']['events'] = total_events
    
    # Сохраняем сводный отчет
    summary_dir = Path("./parsed_events/summary")
    summary_dir.mkdir(parents=True, exist_ok=True)
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"summary_report_{timestamp}.json"
    filepath = summary_dir / filename
    
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(summary_data, f, ensure_ascii=False, indent=2)
    
    size_kb = os.path.getsize(filepath) // 1024
    print(f"  Сводный отчет сохранен: {filepath}")
    print(f"    Размер: {size_kb} KB")
    print(f"    Всего событий: {total_events}")
    print(f"    Обработано городов: {len(all_results)}")

# ============ 7. ГЛАВНАЯ ФУНКЦИЯ ДЛЯ JUPYTER ============
async def main_jupyter():
    """Основная функция для запуска в Jupyter"""
    print(f"\n{'='*70}")
    print("  СИСТЕМА ПАРСИНГА СОБЫТИЙ ДЛЯ JUPYTER")
    print(f"{'='*70}")
    
    try:
        # Выбор городов
        print("\n  Доступные города:")
        print("  1. Москва (msk)")
        print("  2. Санкт-Петербург (spb)")
        print("  3. Оба города")
        
        choice = input("\n  Выберите город(а) для парсинга (1/2/3): ").strip()
        
        if choice == '1':
            cities_to_parse = ['msk']
        elif choice == '2':
            cities_to_parse = ['spb']
        elif choice == '3':
            cities_to_parse = ['msk', 'spb']
        else:
            print("  Неверный выбор, парсим оба города")
            cities_to_parse = ['msk', 'spb']
        
        print(f"\n  Начинаем парсинг {', '.join([CITIES[c]['name'] for c in cities_to_parse])}...")
        
        # Парсинг и экспорт
        all_results = await parse_multiple_cities(cities_to_parse)
        
        # Проверяем результаты
        total_events = sum(len(result.get('events', [])) for result in all_results.values())
        
        if total_events == 0:
            print("\n  Не удалось получить события. Возможные причины:")
            print("   • Нет подключения к интернету")
            print("   • API KudaGo временно недоступно")
            print("   • Нет актуальных событий в выбранных категориях")
            return
        
        # Финальное сообщение
        print(f"\n{'='*70}")
        print("  ВЫПОЛНЕНО УСПЕШНО!")
        print(f"{'='*70}")
        
        print(f"\n  ИТОГОВАЯ СТАТИСТИКА:")
        for city, result in all_results.items():
            events = result.get('events', [])
            if events:
                print(f"    • {CITIES[city]['name']}: {len(events)} событий")
        
        print(f"\n  СОЗДАННЫЕ ФАЙЛЫ И ПАПКИ:")
        print(f"    • parsed_events/ - все экспортированные данные")
        for city in cities_to_parse:
            city_dir = f"./parsed_events/{city}"
            if os.path.exists(city_dir):
                print(f"      ├── {city}/ - данные по {CITIES[city]['name']}")
                print(f"      │   ├── by_city/ - все события города")
                print(f"      │   ├── by_category/ - события по категориям")
                print(f"      │   ├── by_month/ - события по месяцам")
                print(f"      │   └── raw_data/ - сырые данные")
        print(f"      └── summary/ - сводные отчеты")
        
        print(f"\n  ПУТЬ К ДАННЫМ:")
        base_path = os.path.abspath("./parsed_events")
        print(f"    {base_path}")
        
    except KeyboardInterrupt:
        print("\n  Парсинг прерван пользователем")
    except Exception as e:
        print(f"\n  КРИТИЧЕСКАЯ ОШИБКА: {e}")
        import traceback
        traceback.print_exc()

# ============ 8. ЗАПУСК В JUPYTER ============
async def run_in_jupyter():
    """Функция для запуска в Jupyter Notebook"""
    print(f"\n{'='*70}")
    print("  СИСТЕМА ПАРСИНГА СОБЯТИЙ KUDAGO")
    print("  (оптимизировано для Jupyter Notebook)")
    print(f"{'='*70}")
    
    # Создаем папку для данных
    os.makedirs("./parsed_events", exist_ok=True)
    
    try:
        await main_jupyter()
    except Exception as e:
        print(f"\n  Ошибка: {e}")

# ============ 9. ДОПОЛНИТЕЛЬНЫЕ ФУНКЦИИ ============
def check_parsed_data():
    """Проверка экспортированных данных"""
    base_dir = Path("./parsed_events")
    if not base_dir.exists():
        print(" Папка с экспортированными данными не найдена")
        return
    
    print(f"\n СТРУКТУРА ЭКСПОРТИРОВАННЫХ ДАННЫХ:")
    print(f" {base_dir}")
    
    total_files = 0
    total_size_mb = 0
    
    for root, dirs, files in os.walk(base_dir):
        level = root.replace(str(base_dir), '').count(os.sep)
        indent = ' ' * 2 * level
        print(f"{indent}{os.path.basename(root)}/")
        
        subindent = ' ' * 2 * (level + 1)
        for file in files[:5]:  # Показываем первые 5 файлов
            if file.endswith('.json'):
                filepath = os.path.join(root, file)
                size_kb = os.path.getsize(filepath) // 1024
                print(f"{subindent}{file} ({size_kb} KB)")
                total_files += 1
                total_size_mb += size_kb / 1024
        
        if len(files) > 5:
            print(f"{subindent}... и еще {len(files) - 5} файлов")
    
    print(f"\n ИТОГО:")
    print(f"  JSON файлов: {total_files}")
    print(f"  Общий размер: {total_size_mb:.2f} MB")

def load_and_display_event(city: str = 'msk'):
    """Загрузка и отображение примера события"""
    import glob
    
    pattern = f"./parsed_events/{city}/by_city/{city}_all_events_*.json"
    files = glob.glob(pattern)
    
    if not files:
        print(f" Файлы не найдены для города {city}")
        return
    
    latest_file = max(files, key=os.path.getctime)
    
    with open(latest_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    if 'events' in data and data['events']:
        event = data['events'][0]
        print(f"\n ПРИМЕР СОБЫТИЯ:")
        print(f" Название: {event.get('title', 'Нет названия')}")
        print(f" Категория: {event.get('category', 'Нет категории')}")
        print(f" Город: {event.get('city_name', 'Нет города')}")
        
        if 'dates' in event and event['dates']:
            first_date = event['dates'][0]
            if 'start_datetime' in first_date:
                print(f" Дата начала: {first_date['start_datetime']}")
        
        if 'place' in event and event['place']:
            place = event['place']
            print(f" Место: {place.get('title', 'Нет места')}")
            if place.get('address'):
                print(f" Адрес: {place['address']}")
        
        if 'price' in event:
            price = event['price']
            if price.get('is_free'):
                print(f" Цена: Бесплатно")
            elif price.get('min_price'):
                print(f" Цена: от {price['min_price']} {price.get('currency', 'руб.')}")
        
        print(f" Теги: {', '.join(event.get('tags', [])) if event.get('tags') else 'Нет тегов'}")
    else:
        print(" Нет событий в файле")

# ============ 10. ИНТЕРАКТИВНЫЙ ЗАПУСК ============
def setup_jupyter():
    """Настройка для Jupyter Notebook"""
    import nest_asyncio
    nest_asyncio.apply()
    print(" Jupyter настроен для асинхронных операций")
    print("\n Для запуска выполните:")
    print(" await run_in_jupyter()")
    print("\n Или используйте быстрый парсинг:")
    print(" await parse_and_export_city('msk')")

# Автоматическая настройка при импорте
try:
    from IPython import get_ipython
    if get_ipython() is not None:
        setup_jupyter()
except:
    pass

 Проверяем библиотеки...
 Устанавливаем scikit-learn...
 Устанавливаем Pillow...


2025-12-10 23:04:48,196 - INFO - NumExpr defaulting to 12 threads.


 Jupyter настроен для асинхронных операций

 Для запуска выполните:
 await run_in_jupyter()

 Или используйте быстрый парсинг:
 await parse_and_export_city('msk')


In [ ]:
await run_in_jupyter()


  СИСТЕМА ПАРСИНГА СОБЯТИЙ KUDAGO
  (оптимизировано для Jupyter Notebook)

  СИСТЕМА ПАРСИНГА СОБЫТИЙ ДЛЯ JUPYTER

  Доступные города:
  1. Москва (msk)
  2. Санкт-Петербург (spb)
  3. Оба города



  Выберите город(а) для парсинга (1/2/3):  3



  Начинаем парсинг Москва, Санкт-Петербург...

  ЗАПУСК ПАРСИНГА СОБЫТИЙ
 Период: сегодня - конец 2025 года
 Города: Москва, Санкт-Петербург

   НАЧИНАЕМ ПАРСИНГ МОСКВА

 ПАРСИМ СОБЫТИЯ ДЛЯ МОСКВА
  Загружаем Концерты...
